In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/3.model.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Make No'] = df_input['Make No'].astype(int)
df_input['Year'] = df_input['Year'].astype(int)
df_input['Model No'] = df_input['Model No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_request = crawler_series['Model Url']

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    if html.xpath(f'''//b/a[@href="{crawler_series['Model Url']}"]/text()''')[0] == crawler_series['Model Text']:
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if html.xpath('//td[@align="right"]/nobr/font/b/font/b/text()'):
            page = int(html.xpath('//td[@align="right"]/nobr/font/b/font/b/text()')[1].strip().removeprefix('of').strip())
        else:
            print('fuck')

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.append({'Make No': crawler_series['Make No'],
                                      'Make': crawler_series['Make'],
                                      'Year': crawler_series['Year'],
                                      'Model No': crawler_series['Model No'],
                                      'Model': crawler_series['Model'],
                                      'Model Text': crawler_series['Model Text'],
                                      'Model Url': crawler_series['Model Url'],
                                      'Page': page})

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['Make No']}. {crawler_series['Make']} > {crawler_series['Year']} > {crawler_series['Model No']}. {crawler_series['Model']} {url_request}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Make No', 'Year', 'Model No'],
                                                             ascending=[True, False, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/4.page-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Make No', 'Year', 'Model No'],
                                                           ascending=[True, False, True],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/page_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[True] - [请求1次] - [剩余299条] - [2024/11/25 11:39:09] - 51. Toyota > 2010 > 1. 4 Runner https://www.tpms.com/2010-Toyota-4-Runner-TPMS-s/4701.htm

[True] - [请求1次] - [剩余298条] - [2024/11/25 11:39:10] - 51. Toyota > 2010 > 4. Corolla https://www.tpms.com/2010-Toyota-Corolla-TPMS-s/4704.htm

[True] - [请求1次] - [剩余297条] - [2024/11/25 11:39:10] - 51. Toyota > 2010 > 6. Highlander https://www.tpms.com/2010-Toyota-Highlander-TPMS-s/4706.htm

[True] - [请求1次] - [剩余296条] - [2024/11/25 11:39:10] - 51. Toyota > 2010 > 13. Sienna https://www.tpms.com/2010-Toyota-Sienna-TPMS-s/4714.htm

[True] - [请求1次] - [剩余295条] - [2024/11/25 11:39:10] - 51. Toyota > 2010 > 15. Tacoma https://www.tpms.com/2010-Toyota-Tacoma-TPMS-steel-wheels-s/8063.htm

[True] - [请求1次] - [剩余294条] - [2024/11/25 11:39:10] - 51. Toyota > 2010 > 16. Tundra https://www.tpms.com/2010-Toyota-Tundra-TPMS-s/4716.htm

[True] - [请求1次] - [剩余293条] - [2024/11/25 11:39:10] - 51. Toyota > 2010 > 17. Tundra https://www.tpms.com/2010-Toyota-Tundra-TPMS-s